In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## selenium 설치 & 내 드라이브에 chromedriver 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install chromedriver-autoinstaller

#!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' # (최초 1회)

In [ ]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

## 라이브러리 임포트

In [ ]:
import sys
import time
import pandas as pd
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller  # setup chrome options

## chrome_options 설정

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"
sys.path.insert(0,chrome_path)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off : cloab은 새창을 지원하지않기 때문에 창 없는 모드
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

driver = webdriver.Chrome(options=chrome_options)


##유튜브 채널 내 영상 검색 후 url 수집(byCSS_SELECTOR)

####키워드 설정과 검색 및 하나의 영상 url 수집

In [ ]:
# 검색할 채널 & 키워드 설정
channel_name = "@NetflixKorea"
search_keyword = "케이팝데몬헌터스"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# 유튜브 특정 채널 내 검색 URL 생성
url = f"https://www.youtube.com/{channel_name}/search?query={encoded_keyword}"

# 유튜브 검색 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(3)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 페이지 스크롤 다운 (Infinite Scroll)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 하나의 영상 링크 가져오기 (테스트용)
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
titles = [element.text for element in elements if element.text]
if elements:
    test_url = elements[0].get_attribute("href")   #elements[0] : 검색된 영상 중 첫번째 것 가져오기
    print("테스트 URL:", test_url)
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")



####전체 영상 url 수집

In [ ]:
# 모든 영상 제목 및 링크 가져오기
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
results = [element.get_attribute("href") for element in elements if element.get_attribute("href")]

# DataFrame 변환
results_df = pd.DataFrame({'title': titles, 'urls': results})
results_df['video_id'] = results_df['urls'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (현재 디렉토리에 저장됨)
file_name = f"YouTube_Crawling_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/mypython/data/" #본인 구글드라이브의 작업 폴더 경로에 'data' 폴더 만들기
full_path = save_path + file_name  # 전체 파일 경로 생성
results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장

# ChromeDriver 종료
driver.quit()

print(f"🎉 크롤링 완료! 데이터가 '{file_name}'로 저장되었습니다.")

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)

##유튜브 전체 영상 검색 url 수집(byCSS_SELECTOR)

####키워드 설정과 검색 및 하나의 영상 url 수집

In [ ]:
# 검색할 키워드 설정
search_keyword = "케데헌"

# 검색어를 URL 인코딩 (한글, 공백 처리)
encoded_keyword = quote(search_keyword)

# YouTube 전체 검색 URL 생성
url = f"https://www.youtube.com/results?search_query={encoded_keyword}"

# 유튜브 검색 페이지 열기
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
time.sleep(2)

# 페이지가 완전히 로드될 때까지 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

# 페이지 스크롤 다운 (Infinite Scroll)
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 하나의 영상 링크 가져오기 (테스트용)
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
titles = [element.text for element in elements if element.text]
if elements:
    test_url = elements[0].get_attribute("href")  #elements[0] : 검색된 영상 중 첫번째 것 가져오기
    print("테스트 URL:", test_url)
else:
    print("❌ 영상 링크를 찾을 수 없습니다.")


####전체 영상 url 수집

In [ ]:
# 모든 영상 제목 및 링크 가져오기
elements = driver.find_elements(By.CSS_SELECTOR, 'a#video-title')
results = [element.get_attribute("href") for element in elements if element.get_attribute("href")]

# DataFrame 변환
results_df = pd.DataFrame({'title': titles, 'urls': results})
results_df['video_id'] = results_df['urls'].apply(lambda x: x.split('v=')[1].split('&')[0] if x and 'v=' in x else None)

# 결측치 제거 (video_id가 없는 경우 데이터프레임에서 제거)
results_df = results_df.dropna(subset=['video_id']).reset_index(drop=True)

# 엑셀 파일 저장 (현재 디렉토리에 저장됨)
file_name = f"YouTube_Crawling_{search_keyword}.xlsx"
results_df.to_excel(file_name, index=False)

# 엑셀 파일 저장 (지정된 구글 드라이브 경로에 저장)
save_path = "/content/drive/MyDrive/mypython/data/"
full_path = save_path + file_name  # 전체 파일 경로 생성
results_df.to_excel(full_path, index=False)  # 엑셀 파일 저장

# ChromeDriver 종료
driver.quit()

print(f"🎉 크롤링 완료! 데이터가 '{file_name}'로 저장되었습니다.")

# Colab에서 파일 다운로드
from google.colab import files
files.download(file_name)